## Data Exploration
This notebook will allow us to explore the data found in the Kaggle dataset

In [1]:
# imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os

In [17]:
df = pd.read_csv('datasets/cleaned.csv')

In [18]:
df.head()

,Id,Tag,Score_question,Title,Body_question,Score_answer,Body_answer
0,469,"['python', 'osx', 'fonts', 'photoshop']",21,find full path font display name mac,using photoshop javascript api find font given...,4,open terminal terminal type locate insertfonth...
1,469,"['python', 'osx', 'fonts', 'photoshop']",21,find full path font display name mac,using photoshop javascript api find font given...,2,not able find anything directly think iterate ...
2,469,"['python', 'osx', 'fonts', 'photoshop']",21,find full path font display name mac,using photoshop javascript api find font given...,12,unfortunately api not deprecated located appli...
3,469,"['python', 'osx', 'fonts', 'photoshop']",21,find full path font display name mac,using photoshop javascript api find font given...,1,must method cocoa get list font would use pyob...
4,502,"['python', 'windows', 'image', 'pdf']",27,get preview jpeg pdf window,python application need generate jpeg preview ...,9,use imagemagick convert utility see example ht...


In [21]:
df["Body_answer"].apply(lambda x: str(x) + " ")

0         open terminal terminal type locate insertfonth...
1         not able find anything directly think iterate ...
2         unfortunately api not deprecated located appli...
3         must method cocoa get list font would use pyob...
4         use imagemagick convert utility see example ht...
                                ...                        
987117        quick dirty way def x st x return call hello 
987118    may check value variable would none nothing fo...
987119    first thing use instead separate block conside...
987120                                          use python 
987121              tank answer found answer bash code cat 
Name: Body_answer, Length: 987122, dtype: object

In [22]:
combined_answers = df.groupby("Id")["Body_answer"].sum().to_frame()
combined_answers.head(3)

,Body_answer
Id,
469,open terminal terminal type locate insertfonth...
502,use imagemagick convert utility see example ht...
535,one possibility hudson written java integratio...


In [23]:
questions = df[["Id", "Body_question"]]
questions.head(3)

,Id,Body_question
0,469,using photoshop javascript api find font given...
1,469,using photoshop javascript api find font given...
2,469,using photoshop javascript api find font given...


In [28]:
combined = questions.merge(combined_answers, left_on="Id", right_index=True)
combined = combined["Body_question"].astype(str) + " " + combined["Body_answer"].astype(str)
combined = combined.drop_duplicates()

In [30]:
combined.head(3)

0    using photoshop javascript api find font given...
4    python application need generate jpeg preview ...
7    starting work hobby project python codebase wo...
dtype: object

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer()
subset = combined.iloc[:100] # fit on full dataset overnight
vectorizer.fit(subset)

,input,'content'
,encoding,'utf-8'
,decode_error,'strict'
,strip_accents,None
,lowercase,True
,preprocessor,None
,tokenizer,None
,analyzer,'word'
,stop_words,None
,token_pattern,'(?u)\\b\\w\\w+\\b'
,ngram_range,"(1, ...)"


In [39]:
vectorizer.transform(subset)

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 16097 stored elements and shape (100, 4762)>

In [58]:
query = df.iloc[4]["Body_question"]
query

'python application need generate jpeg preview first page pdf mac spawning sip something similarly simple window'

Consider querying by tags for a query speed up instead of going through all Stack posts

In [62]:
combined.shape

(539237,)

In [63]:
def model(query):
    tfidf_sim = vectorizer.transform([query])
    scores = np.dot(vectorizer.transform(combined.iloc[:1000]), tfidf_sim.T)
    best_post_ix = np.argmax(scores)
    best_post = combined.iloc[best_post_ix]
    
    return best_post

model(query)

'python application need generate jpeg preview first page pdf mac spawning sip something similarly simple window use imagemagick convert utility see example http convert convert two page pdf file jpeg file also convert jpegs thumbnail follows convert even convert pdf directly jpeg thumbnail follows convert result two pagepc likely acrobat installed think acrobat installs shell extension preview first page pdf document appear window explorer thumbnail view get thumbnail via iextractimage com api need wrap vbaccelerator example c could port pythonimagemagick delegate bitmap conversion ghostscript anyway command use based actual command listed p alpha delegate imagemagick adjusted use jpeg output g output input output input output input filename adjust whatever resolution need obviously strip backslashes writing whole command one line good two reason not need imagemagick installed anymore not anything imagemagick love bit believe simple solution imagemagick conversion first ppm jpeg way c

In [75]:
import plotly.express as px
to_plot = df.iloc[:10000,:]["Body_answer"].fillna("").astype(str).apply(lambda x: vectorizer.transform([str(x)]).sum())

px.histogram(to_plot)